In [67]:
import numpy as np
import pandas as pd
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from librosa import display
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [68]:
df = pd.read_csv(r"C:\Users\anurag\Desktop\first_jupyter\esc50.csv")
labels = list(df['category'].unique())
df.head()


,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [69]:
ground_location = r"C:\Users\anurag\Desktop\first_jupyter\audio"
audio_path = os.path.join(ground_location, 'audio')

In [73]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0) # No need for assignment
    return mfccs_scaled_features

In [74]:
df['extracted_feature'] = df['filename'].apply(lambda x: features_extractor(os.path.join(audio_path, x)))



In [75]:
df.head()

,filename,fold,target,category,esc10,src_file,take,extracted_feature
0,1-100032-A-0.wav,1,0,dog,True,100032,A,"[-600.9402, 4.6942906, -8.5059395, -4.276621, ..."
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,"[-193.9782, 2.746148, -59.386883, 4.4134607, -..."
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,"[16.259266, 56.921246, -8.960626, 15.122985, -..."
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,"[17.771097, 54.86499, -7.1793256, 13.654805, -..."
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,"[-423.07523, 124.92982, 38.76069, 35.04086, -2..."


In [76]:
selected_columns = ['extracted_feature', 'category']
new_df = df[selected_columns]
new_df = new_df.rename(columns={"extracted_feature": "feature", "category": "class"})
print(new_df.head())

                                             feature           class
0  [-600.9402, 4.6942906, -8.5059395, -4.276621, ...             dog
1  [-193.9782, 2.746148, -59.386883, 4.4134607, -...  chirping_birds
2  [16.259266, 56.921246, -8.960626, 15.122985, -...  vacuum_cleaner
3  [17.771097, 54.86499, -7.1793256, 13.654805, -...  vacuum_cleaner
4  [-423.07523, 124.92982, 38.76069, 35.04086, -2...    thunderstorm


In [77]:
X=np.array(new_df['feature'].tolist())
y=np.array(new_df['class'].tolist())

In [78]:
X.shape

(2000, 40)

In [79]:
y = np.array(pd.get_dummies(y))

In [80]:
y.shape

(2000, 50)

In [81]:

from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state = 40)

In [82]:
X_train.shape

(1600, 40)

In [83]:
X_train

array([[-3.9559586e+02,  2.4499111e+01, -6.2602417e+01, ...,
         3.4795412e-01, -8.4258325e-02,  4.2547783e-01],
       [-3.4597650e+02,  7.3021889e+01, -3.9317745e-01, ...,
         2.8718266e-01, -2.3896637e+00,  1.5628420e+00],
       [-2.7734686e+02,  7.2951607e+01, -5.6011128e+00, ...,
         7.7596772e-01,  3.1774712e-01,  2.5888455e+00],
       ...,
       [-4.8884232e+02,  1.1344060e+01,  4.6376405e+00, ...,
         2.1811275e+00,  7.0965037e-02,  3.5240066e+00],
       [-1.6922687e+02,  3.7419380e+01, -2.7616776e+01, ...,
         7.2553068e-01, -4.9324974e-01,  8.7610066e-02],
       [ 1.7776417e+01,  1.9014933e+01, -2.0101736e+01, ...,
         7.4992257e-01, -1.7313024e+00, -2.6366544e-01]], dtype=float32)

In [84]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

In [86]:
num_labels = y.shape[1]
num_labels

50

In [87]:
model = Sequential()

# First Convolutional Layer
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(40, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Second Convolutional Layer
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Third Convolutional Layer
model.add(Conv1D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Flattening the output
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [88]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)                    │ (None, 38, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 17, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 6, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_7 (MaxPooling1D)       │ (None, 3, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 3, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 50)                  │           6,450 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 228,402 (892.20 KB)

 Trainable params: 228,402 (892.20 KB)

 Non-trainable params: 0 (0.00 B)

In [89]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [90]:
num_epoch = 58
num_batch = 32
 
model.fit(X_train, y_train, batch_size=num_batch, epochs=num_epoch, validation_data=(X_test, y_test),verbose=1)


#duration = datetime.now() - start
#print("Training completed in time: ", duration)

Epoch 1/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0218 - loss: 5.3389 - val_accuracy: 0.0475 - val_loss: 3.8768
Epoch 2/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0274 - loss: 3.8862 - val_accuracy: 0.0775 - val_loss: 3.8115
Epoch 3/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0242 - loss: 3.8537 - val_accuracy: 0.0700 - val_loss: 3.7476
Epoch 4/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0560 - loss: 3.7586 - val_accuracy: 0.0800 - val_loss: 3.6457
Epoch 5/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0824 - loss: 3.7175 - val_accuracy: 0.1375 - val_loss: 3.5193
Epoch 6/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0870 - loss: 3.6540 - val_accuracy: 0.1725 - val_loss: 3.4348
Epoch 7/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0865 - loss: 3.5543 - val_accuracy: 0.1800 - val_loss: 3.2845
Epoch 8/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1142 - loss: 3.4292 - val_accuracy: 0.2050 - val_loss:

In [60]:
test_accuracy = model.evaluate(X_test,y_test,verbose =0)
print(test_accuracy[1])

0.017500000074505806


In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
model_2 = Sequential()
model_2.add(Conv1D(64, 3, padding="same", activation="relu", input_shape=(40, 1)))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(Conv1D(128, 3, padding="same", activation="relu"))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(Dropout(0.1))
model_2.add(Flatten())
model_2.add(Dense(1024, activation="sigmoid"))
model_2.add(Dense(50, activation="softmax"))

model_2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 40, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 20, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 50)                  │          51,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,387,954 (5.29 MB)

 Trainable params: 1,387,954 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
model_2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_2.fit(X_train, y_train, batch_size=num_batch, epochs=num_epoch, validation_data=(X_test, y_test), verbose=1)

Epoch 1/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0795 - loss: 3.8951 - val_accuracy: 0.2100 - val_loss: 2.9657
Epoch 2/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2334 - loss: 2.8152 - val_accuracy: 0.3150 - val_loss: 2.4868
Epoch 3/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3815 - loss: 2.2631 - val_accuracy: 0.3650 - val_loss: 2.1814
Epoch 4/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4899 - loss: 1.7653 - val_accuracy: 0.3500 - val_loss: 2.1855
Epoch 5/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6003 - loss: 1.4391 - val_accuracy: 0.4325 - val_loss: 1.9244
Epoch 6/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6607 - loss: 1.2329 - val_accuracy: 0.4650 - val_loss: 1.8229
Epoch 7/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7331 - loss: 1.0167 - val_accuracy: 0.4875 - val_loss: 1.7715
Epoch 8/58
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7978 - loss: 0.7896 - val_accuracy: 0.5125 - val_loss

In [91]:
test_accuracy = model_2.evaluate(X_test,y_test,verbose =0)
print(test_accuracy[1])

0.5600000023841858
